In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import torch.nn.functional as F
import numpy as np

In [2]:
NB_CLASS=59
BATCH_SIZE=32
IMAGE_SIZE=299
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json'
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')
    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.inceptionresnetv2_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model


def train(epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionResnet/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.79
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3 or epoch==4 or epoch==5:
            lr=0.00006
            momentum=0.95
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if epoch==6:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/Inception_Resnet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))    
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1           
           
        if accuracy>min_acc:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)
  

In [ ]:
#train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.093750,loss:4.005427
[epoch:0,batch:59]:acc: 0.170313,loss:3.775710
[epoch:0,batch:89]:acc: 0.232639,loss:3.556760
[epoch:0,batch:119]:acc: 0.278385,loss:3.398135
[epoch:0,batch:149]:acc: 0.315625,loss:3.253402
[epoch:0,batch:179]:acc: 0.342535,loss:3.137309
[epoch:0,batch:209]:acc: 0.366964,loss:3.029435
[epoch:0,batch:239]:acc: 0.388542,loss:2.928768
[epoch:0,batch:269]:acc: 0.405208,loss:2.845991
[epoch:0,batch:299]:acc: 0.423021,loss:2.764742
[epoch:0,batch:299]: val_loss:1.861431,val_acc:0.625688,val_total:4539
[epoch:0,batch:329]:acc: 0.434659,loss:2.694619
[epoch:0,batch:359]:acc: 0.444878,loss:2.631458
[epoch:0,batch:389]:acc: 0.453205,loss:2.572280
[epoch:0,batch:419]:acc: 0.461086,loss:2.517458
[epoch:0,batch:449]:acc: 0.469583,loss:2.466710
[epoch:0,batch:479]:acc: 0.476432,loss:2.420923
[epoch:0,batch:509]:acc: 0.483456,loss:2.375575
[epoch:0,batch:539]:acc: 0.490451,loss:2.33249

[epoch:4,batch:89]:acc: 0.825000,loss:0.455184
[epoch:4,batch:119]:acc: 0.826042,loss:0.443579
[epoch:4,batch:149]:acc: 0.827708,loss:0.436180
[epoch:4,batch:179]:acc: 0.826910,loss:0.432538
[epoch:4,batch:209]:acc: 0.826190,loss:0.434145
[epoch:4,batch:239]:acc: 0.826042,loss:0.434112
[epoch:4,batch:269]:acc: 0.827083,loss:0.434377
[epoch:4,batch:299]:acc: 0.824063,loss:0.439270
[epoch:4,batch:299]: val_loss:0.459238,val_acc:0.831901,val_total:4539
[epoch:4,batch:329]:acc: 0.824621,loss:0.436190
[epoch:4,batch:359]:acc: 0.824740,loss:0.437097
[epoch:4,batch:389]:acc: 0.824679,loss:0.437036
[epoch:4,batch:419]:acc: 0.823884,loss:0.435798
[epoch:4,batch:449]:acc: 0.824583,loss:0.435435
[epoch:4,batch:479]:acc: 0.824479,loss:0.434677
[epoch:4,batch:509]:acc: 0.825613,loss:0.435147
[epoch:4,batch:539]:acc: 0.826042,loss:0.435232
[epoch:4,batch:569]:acc: 0.826974,loss:0.434657
[epoch:4,batch:599]:acc: 0.828490,loss:0.434623
[epoch:4,batch:599]: val_loss:0.431953,val_acc:0.829258,val_total:

[epoch:8,batch:179]:acc: 0.868924,loss:0.317320
[epoch:8,batch:209]:acc: 0.867262,loss:0.320516
[epoch:8,batch:239]:acc: 0.869661,loss:0.318395
[epoch:8,batch:269]:acc: 0.871412,loss:0.315479
[epoch:8,batch:299]:acc: 0.868333,loss:0.317659
[epoch:8,batch:299]: val_loss:0.439586,val_acc:0.835647,val_total:4539
[epoch:8,batch:329]:acc: 0.868277,loss:0.318166
[epoch:8,batch:359]:acc: 0.868403,loss:0.318992
[epoch:8,batch:389]:acc: 0.866506,loss:0.325194
[epoch:8,batch:419]:acc: 0.866071,loss:0.323837
[epoch:8,batch:449]:acc: 0.865278,loss:0.326087
[epoch:8,batch:479]:acc: 0.865299,loss:0.328054
[epoch:8,batch:509]:acc: 0.864093,loss:0.329817
[epoch:8,batch:539]:acc: 0.864699,loss:0.327765
[epoch:8,batch:569]:acc: 0.864638,loss:0.326513
[epoch:8,batch:599]:acc: 0.865365,loss:0.324380
[epoch:8,batch:599]: val_loss:0.439744,val_acc:0.836748,val_total:4539
[epoch:8,batch:629]:acc: 0.865476,loss:0.324123
[epoch:8,batch:659]:acc: 0.864867,loss:0.325450
[epoch:8,batch:689]:acc: 0.864629,loss:0.3

[epoch:12,batch:239]:acc: 0.896094,loss:0.236634
[epoch:12,batch:269]:acc: 0.896528,loss:0.236111
[epoch:12,batch:299]:acc: 0.895833,loss:0.232873
[epoch:12,batch:299]: val_loss:0.398982,val_acc:0.860762,val_total:4539
[epoch:12,batch:329]:acc: 0.895455,loss:0.233710
[epoch:12,batch:359]:acc: 0.895486,loss:0.233710
[epoch:12,batch:389]:acc: 0.895673,loss:0.233214
[epoch:12,batch:419]:acc: 0.895908,loss:0.232737
[epoch:12,batch:449]:acc: 0.895694,loss:0.233951
[epoch:12,batch:479]:acc: 0.895768,loss:0.233849
[epoch:12,batch:509]:acc: 0.896936,loss:0.231674
[epoch:12,batch:539]:acc: 0.897164,loss:0.230593
[epoch:12,batch:569]:acc: 0.897039,loss:0.230952
[epoch:12,batch:599]:acc: 0.896094,loss:0.231715
[epoch:12,batch:599]: val_loss:0.400974,val_acc:0.862084,val_total:4539
[epoch:12,batch:629]:acc: 0.896181,loss:0.232161
[epoch:12,batch:659]:acc: 0.896307,loss:0.231332
[epoch:12,batch:689]:acc: 0.896558,loss:0.230806
[epoch:12,batch:719]:acc: 0.896398,loss:0.230241
[epoch:12,batch:749]:ac

[epoch:16,batch:329]:acc: 0.903504,loss:0.227946
[epoch:16,batch:359]:acc: 0.902344,loss:0.228896
[epoch:16,batch:389]:acc: 0.901202,loss:0.229806
[epoch:16,batch:419]:acc: 0.900595,loss:0.229469
[epoch:16,batch:449]:acc: 0.900139,loss:0.231385
[epoch:16,batch:479]:acc: 0.899219,loss:0.232654
[epoch:16,batch:509]:acc: 0.899020,loss:0.234564
[epoch:16,batch:539]:acc: 0.898900,loss:0.233681
[epoch:16,batch:569]:acc: 0.898958,loss:0.233537
[epoch:16,batch:599]:acc: 0.898281,loss:0.233802
[epoch:16,batch:599]: val_loss:0.399929,val_acc:0.860983,val_total:4539
[epoch:16,batch:629]:acc: 0.897470,loss:0.234184
[epoch:16,batch:659]:acc: 0.897964,loss:0.232560
[epoch:16,batch:689]:acc: 0.898777,loss:0.231208
[epoch:16,batch:719]:acc: 0.899349,loss:0.229949
[epoch:16,batch:749]:acc: 0.899500,loss:0.229570
[epoch:16,batch:779]:acc: 0.899800,loss:0.228706
[epoch:16,batch:809]:acc: 0.900039,loss:0.228029
[epoch:16,batch:839]:acc: 0.900893,loss:0.227150
[epoch:16,batch:869]:acc: 0.900108,loss:0.2276

In [19]:
# modelParams=torch.load('../model/Inception_Resnet/2018-10-29_loss_best.pth')
# # optimizerNew=torch.optim.SGD(getmodel().parameters(),lr=0.001,momentum=0.9)
# optimizerNew=modelParams['optimizer']
# for paramgroup in optimizerNew.param_groups:
#     print(paramgroup)

{'state': {140396681407512: {'momentum_buffer': tensor([[[[ 1.0612e-02,  8.8870e-03,  5.9221e-03],
          [ 8.3318e-03,  4.4422e-03,  8.1511e-03],
          [ 1.2735e-02, -5.7604e-04,  9.1059e-03]],

         [[ 4.6710e-03,  2.3269e-03, -1.2687e-03],
          [ 2.0623e-03, -4.8352e-03, -2.7741e-03],
          [ 3.1410e-03, -1.0409e-02, -1.6593e-03]],

         [[ 4.2111e-03,  1.3880e-03, -9.6481e-04],
          [ 7.0282e-04, -5.7044e-03, -2.4694e-03],
          [ 2.9737e-03, -9.8794e-03, -7.3623e-04]]],


        [[[-1.1762e-02, -4.3675e-03, -3.1087e-03],
          [-1.0226e-02, -1.9647e-03, -2.1038e-04],
          [-1.0949e-02, -2.4742e-03, -8.4319e-04]],

         [[-1.4819e-02, -7.6228e-03, -6.4292e-03],
          [-1.3328e-02, -5.6633e-03, -3.2213e-03],
          [-1.4208e-02, -5.8413e-03, -3.9824e-03]],

         [[-1.1634e-02, -3.5144e-03, -1.4249e-03],
          [-1.0324e-02, -2.5722e-03, -1.2092e-04],
          [-1.0731e-02, -1.5271e-03, -5.6143e-04]]],


        [[[-5.0944

In [4]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionResnet/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))   
    min_acc=max(modelParams['val_correct'],0.81)
    print('val_correct is %f'%(min_acc))
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/Inception_Resnet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
        optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum,weight_decay=1e-4)
        print('lr now is %f'%(lr))
        print('now patience is %d '%(patience))
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))      
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
reuseTrain('../model/Inception_Resnet/2018-10-31_acc_best.pth',80)

[+] loading model... Done
min_loss is :0.423642
val_correct is 0.836748
Epoch 28/79
----------
lr now is 0.000100
now patience is 0 
[epoch:28,batch:29]:acc: 0.839583,loss:0.413527
[epoch:28,batch:59]:acc: 0.839583,loss:0.395806
[epoch:28,batch:89]:acc: 0.836458,loss:0.397394
[epoch:28,batch:119]:acc: 0.837500,loss:0.395699
[epoch:28,batch:149]:acc: 0.839792,loss:0.392403
[epoch:28,batch:179]:acc: 0.844444,loss:0.386421
[epoch:28,batch:209]:acc: 0.839881,loss:0.387968
[epoch:28,batch:239]:acc: 0.841276,loss:0.384777
[epoch:28,batch:269]:acc: 0.841435,loss:0.388705
[epoch:28,batch:299]:acc: 0.840313,loss:0.389065
[epoch:28,batch:299]: val_loss:0.391917,val_acc:0.837409,val_total:4539
[epoch:28,batch:329]:acc: 0.839773,loss:0.390960
[epoch:28,batch:359]:acc: 0.840799,loss:0.386750
[epoch:28,batch:389]:acc: 0.838862,loss:0.388422
[epoch:28,batch:419]:acc: 0.839360,loss:0.388067
[epoch:28,batch:449]:acc: 0.839097,loss:0.386919
[epoch:28,batch:479]:acc: 0.838346,loss:0.387411
[epoch:28,batc

[epoch:31] :acc: 0.849008,loss:0.369347,lr:0.000100,patience:1
[epoch:31]: val_loss:0.384776,val_acc:0.842256,
Epoch 32/79
----------
lr now is 0.000010
now patience is 0 
[epoch:32,batch:29]:acc: 0.823958,loss:0.402859
[epoch:32,batch:59]:acc: 0.834375,loss:0.384331
[epoch:32,batch:89]:acc: 0.841667,loss:0.372807
[epoch:32,batch:119]:acc: 0.843490,loss:0.377051
[epoch:32,batch:149]:acc: 0.847708,loss:0.375553
[epoch:32,batch:179]:acc: 0.845139,loss:0.379009
[epoch:32,batch:209]:acc: 0.845238,loss:0.378482
[epoch:32,batch:239]:acc: 0.844792,loss:0.378393
[epoch:32,batch:269]:acc: 0.843519,loss:0.379336
[epoch:32,batch:299]:acc: 0.843854,loss:0.378587
[epoch:32,batch:299]: val_loss:0.385920,val_acc:0.842036,val_total:4539
[epoch:32,batch:329]:acc: 0.844508,loss:0.379155
[epoch:32,batch:359]:acc: 0.844271,loss:0.379432
[epoch:32,batch:389]:acc: 0.845994,loss:0.377878
[epoch:32,batch:419]:acc: 0.846429,loss:0.377576
[epoch:32,batch:449]:acc: 0.847431,loss:0.377319
[epoch:32,batch:479]:acc

[epoch:36,batch:59]:acc: 0.832292,loss:0.388897
[epoch:36,batch:89]:acc: 0.840972,loss:0.368213
[epoch:36,batch:119]:acc: 0.840365,loss:0.370161
[epoch:36,batch:149]:acc: 0.840000,loss:0.372666
[epoch:36,batch:179]:acc: 0.844792,loss:0.368972
[epoch:36,batch:209]:acc: 0.847321,loss:0.365869
[epoch:36,batch:239]:acc: 0.848828,loss:0.366121
[epoch:36,batch:269]:acc: 0.849421,loss:0.364596
[epoch:36,batch:299]:acc: 0.851250,loss:0.363775
[epoch:36,batch:299]: val_loss:0.383803,val_acc:0.844018,val_total:4539
[epoch:36,batch:329]:acc: 0.850568,loss:0.367624
[epoch:36,batch:359]:acc: 0.849740,loss:0.367590
[epoch:36,batch:389]:acc: 0.849279,loss:0.368959
[epoch:36,batch:419]:acc: 0.850000,loss:0.367551
[epoch:36,batch:449]:acc: 0.850417,loss:0.368488
[epoch:36,batch:479]:acc: 0.851693,loss:0.365925
[epoch:36,batch:509]:acc: 0.851654,loss:0.366346
[epoch:36,batch:539]:acc: 0.850521,loss:0.366464
[epoch:36,batch:569]:acc: 0.850000,loss:0.368502
[epoch:36,batch:599]:acc: 0.850052,loss:0.367941

In [4]:
def TrainWithRawData(path,epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionResnet/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/Inception_Resnet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            
         #   optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
         #   optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'Inception_Resnet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'Inception_Resnet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/Inception_Resnet/2018-11-02_loss_best.pth',80)

[+] loading model... Done
min_loss is :0.363586
val_correct is 0.862965
Epoch 17/79
----------
begin lr is  0.0001
[epoch:17,batch:29]:acc: 0.903125,loss:0.219083
[epoch:17,batch:59]:acc: 0.900521,loss:0.227394
[epoch:17,batch:89]:acc: 0.901736,loss:0.229610
[epoch:17,batch:119]:acc: 0.905208,loss:0.223892
[epoch:17,batch:149]:acc: 0.903333,loss:0.223685
[epoch:17,batch:179]:acc: 0.906250,loss:0.218032
[epoch:17,batch:209]:acc: 0.906696,loss:0.219775
[epoch:17,batch:239]:acc: 0.905859,loss:0.219434
[epoch:17,batch:269]:acc: 0.906713,loss:0.216971
[epoch:17,batch:299]:acc: 0.907917,loss:0.216959
[epoch:17,batch:299]: val_loss:0.363270,val_acc:0.862745,val_total:4539
save new model loss,now loss is  0.36327049136161804
[epoch:17,batch:329]:acc: 0.908144,loss:0.217298
[epoch:17,batch:359]:acc: 0.907899,loss:0.216900
[epoch:17,batch:389]:acc: 0.907612,loss:0.218392
[epoch:17,batch:419]:acc: 0.908110,loss:0.217827
[epoch:17,batch:449]:acc: 0.908542,loss:0.217510
[epoch:17,batch:479]:acc: 0.

[epoch:20,batch:959]:acc: 0.907780,loss:0.211187
[epoch:20,batch:989]:acc: 0.907670,loss:0.211536
[epoch:20] :acc: 0.907715,loss:0.211494,lr:0.000020,patience:1
[epoch:20]: val_loss:0.367583,val_acc:0.862084,
Epoch 21/79
----------
lr desencd
[epoch:21,batch:29]:acc: 0.903125,loss:0.235355
[epoch:21,batch:59]:acc: 0.904687,loss:0.225887
[epoch:21,batch:89]:acc: 0.905556,loss:0.218670
[epoch:21,batch:119]:acc: 0.908854,loss:0.214807
[epoch:21,batch:149]:acc: 0.909792,loss:0.215043
[epoch:21,batch:179]:acc: 0.910937,loss:0.213055
[epoch:21,batch:209]:acc: 0.910863,loss:0.211201
[epoch:21,batch:239]:acc: 0.910026,loss:0.215619
[epoch:21,batch:269]:acc: 0.911111,loss:0.213887
[epoch:21,batch:299]:acc: 0.912188,loss:0.212374
[epoch:21,batch:299]: val_loss:0.360109,val_acc:0.864067,val_total:4539
[epoch:21,batch:329]:acc: 0.911837,loss:0.213133
[epoch:21,batch:359]:acc: 0.912066,loss:0.211947
[epoch:21,batch:389]:acc: 0.911058,loss:0.211963
[epoch:21,batch:419]:acc: 0.910863,loss:0.212325
[e

[epoch:25,batch:89]:acc: 0.900694,loss:0.220911
[epoch:25,batch:119]:acc: 0.903385,loss:0.219181
[epoch:25,batch:149]:acc: 0.903750,loss:0.214902
[epoch:25,batch:179]:acc: 0.905208,loss:0.213265
[epoch:25,batch:209]:acc: 0.904315,loss:0.212952
[epoch:25,batch:239]:acc: 0.905599,loss:0.212620
[epoch:25,batch:269]:acc: 0.905671,loss:0.212617
[epoch:25,batch:299]:acc: 0.905625,loss:0.212225
[epoch:25,batch:299]: val_loss:0.361119,val_acc:0.865169,val_total:4539
[epoch:25,batch:329]:acc: 0.905871,loss:0.212025
[epoch:25,batch:359]:acc: 0.905642,loss:0.213480
[epoch:25,batch:389]:acc: 0.906410,loss:0.214327
[epoch:25,batch:419]:acc: 0.906696,loss:0.213765
[epoch:25,batch:449]:acc: 0.906806,loss:0.215427
[epoch:25,batch:479]:acc: 0.906771,loss:0.214730
[epoch:25,batch:509]:acc: 0.907414,loss:0.214353
[epoch:25,batch:539]:acc: 0.907234,loss:0.215310
[epoch:25,batch:569]:acc: 0.908169,loss:0.214020
[epoch:25,batch:599]:acc: 0.907708,loss:0.214201
[epoch:25,batch:599]: val_loss:0.359725,val_acc

[epoch:29,batch:299]:acc: 0.907813,loss:0.211219
[epoch:29,batch:299]: val_loss:0.361808,val_acc:0.863406,val_total:4539
[epoch:29,batch:329]:acc: 0.908239,loss:0.211300
[epoch:29,batch:359]:acc: 0.907031,loss:0.212932
[epoch:29,batch:389]:acc: 0.907212,loss:0.212752
[epoch:29,batch:419]:acc: 0.907738,loss:0.211627
[epoch:29,batch:449]:acc: 0.907708,loss:0.212464
[epoch:29,batch:479]:acc: 0.907031,loss:0.214354
[epoch:29,batch:509]:acc: 0.907475,loss:0.213817
[epoch:29,batch:539]:acc: 0.907350,loss:0.213852
[epoch:29,batch:569]:acc: 0.907566,loss:0.213849
[epoch:29,batch:599]:acc: 0.907396,loss:0.214301
[epoch:29,batch:599]: val_loss:0.362698,val_acc:0.863186,val_total:4539
[epoch:29,batch:629]:acc: 0.908135,loss:0.212803
[epoch:29,batch:659]:acc: 0.906913,loss:0.215489
[epoch:29,batch:689]:acc: 0.906567,loss:0.215598
[epoch:29,batch:719]:acc: 0.907161,loss:0.214880
[epoch:29,batch:749]:acc: 0.907333,loss:0.215000
[epoch:29,batch:779]:acc: 0.907372,loss:0.214736
[epoch:29,batch:809]:ac

[epoch:33,batch:449]:acc: 0.907708,loss:0.214858
[epoch:33,batch:479]:acc: 0.908073,loss:0.214558
[epoch:33,batch:509]:acc: 0.908640,loss:0.213885
[epoch:33,batch:539]:acc: 0.908333,loss:0.213624
[epoch:33,batch:569]:acc: 0.908004,loss:0.214000
[epoch:33,batch:599]:acc: 0.907552,loss:0.214694
[epoch:33,batch:599]: val_loss:0.363473,val_acc:0.863626,val_total:4539
[epoch:33,batch:629]:acc: 0.907292,loss:0.214900
[epoch:33,batch:659]:acc: 0.907386,loss:0.214431
[epoch:33,batch:689]:acc: 0.907790,loss:0.213973
[epoch:33,batch:719]:acc: 0.907726,loss:0.214221
[epoch:33,batch:749]:acc: 0.907917,loss:0.213976
[epoch:33,batch:779]:acc: 0.907933,loss:0.213635
[epoch:33,batch:809]:acc: 0.907562,loss:0.214362
[epoch:33,batch:839]:acc: 0.907626,loss:0.214000
[epoch:33,batch:869]:acc: 0.907435,loss:0.214159
[epoch:33,batch:899]:acc: 0.907778,loss:0.213493
[epoch:33,batch:899]: val_loss:0.359303,val_acc:0.864728,val_total:4539
[epoch:33,batch:929]:acc: 0.907762,loss:0.213344
[epoch:33,batch:959]:ac

[epoch:37,batch:629]:acc: 0.908978,loss:0.213460
[epoch:37,batch:659]:acc: 0.909422,loss:0.213296
[epoch:37,batch:689]:acc: 0.909873,loss:0.213410
[epoch:37,batch:719]:acc: 0.909332,loss:0.214168
[epoch:37,batch:749]:acc: 0.909292,loss:0.214086
[epoch:37,batch:779]:acc: 0.909175,loss:0.214141
[epoch:37,batch:809]:acc: 0.908796,loss:0.214248
[epoch:37,batch:839]:acc: 0.909263,loss:0.213530
[epoch:37,batch:869]:acc: 0.908405,loss:0.214096
[epoch:37,batch:899]:acc: 0.908924,loss:0.213133
[epoch:37,batch:899]: val_loss:0.361677,val_acc:0.864287,val_total:4539
[epoch:37,batch:929]:acc: 0.909140,loss:0.212663
[epoch:37,batch:959]:acc: 0.909277,loss:0.212070
[epoch:37,batch:989]:acc: 0.909343,loss:0.212019
[epoch:37] :acc: 0.909197,loss:0.212139,lr:0.000000,patience:0
[epoch:37]: val_loss:0.365451,val_acc:0.862525,
Epoch 38/79
----------
[epoch:38,batch:29]:acc: 0.911458,loss:0.203731
[epoch:38,batch:59]:acc: 0.911979,loss:0.201264
[epoch:38,batch:89]:acc: 0.913889,loss:0.201127
[epoch:38,bat

[epoch:41,batch:809]:acc: 0.908102,loss:0.210793
[epoch:41,batch:839]:acc: 0.908147,loss:0.211618
[epoch:41,batch:869]:acc: 0.908441,loss:0.211241
[epoch:41,batch:899]:acc: 0.908542,loss:0.210979
[epoch:41,batch:899]: val_loss:0.362605,val_acc:0.862525,val_total:4539
[epoch:41,batch:929]:acc: 0.908804,loss:0.209761
[epoch:41,batch:959]:acc: 0.909049,loss:0.209780
[epoch:41,batch:989]:acc: 0.908902,loss:0.209724
[epoch:41] :acc: 0.908913,loss:0.210074,lr:0.000000,patience:0
[epoch:41]: val_loss:0.367866,val_acc:0.862525,
Epoch 42/79
----------
[epoch:42,batch:29]:acc: 0.900000,loss:0.225526
[epoch:42,batch:59]:acc: 0.900000,loss:0.236230
[epoch:42,batch:89]:acc: 0.902083,loss:0.225729
[epoch:42,batch:119]:acc: 0.903385,loss:0.226773
[epoch:42,batch:149]:acc: 0.907917,loss:0.219866
[epoch:42,batch:179]:acc: 0.910243,loss:0.212751
[epoch:42,batch:209]:acc: 0.909524,loss:0.212906
[epoch:42,batch:239]:acc: 0.911849,loss:0.209232
[epoch:42,batch:269]:acc: 0.912731,loss:0.206895
[epoch:42,bat

[epoch:45,batch:959]:acc: 0.909993,loss:0.212035
[epoch:45,batch:989]:acc: 0.909848,loss:0.211832
[epoch:45] :acc: 0.909828,loss:0.212316,lr:0.000000,patience:0
[epoch:45]: val_loss:0.368071,val_acc:0.863406,
Epoch 46/79
----------
[epoch:46,batch:29]:acc: 0.912500,loss:0.223256
[epoch:46,batch:59]:acc: 0.908333,loss:0.222318
[epoch:46,batch:89]:acc: 0.908681,loss:0.219175
[epoch:46,batch:119]:acc: 0.912240,loss:0.212279
[epoch:46,batch:149]:acc: 0.912083,loss:0.210762
[epoch:46,batch:179]:acc: 0.914757,loss:0.205709
[epoch:46,batch:209]:acc: 0.914583,loss:0.207942
[epoch:46,batch:239]:acc: 0.912370,loss:0.211392
[epoch:46,batch:269]:acc: 0.912153,loss:0.209754
[epoch:46,batch:299]:acc: 0.912292,loss:0.209619
[epoch:46,batch:299]: val_loss:0.356762,val_acc:0.863406,val_total:4539
save new model loss,now loss is  0.3567620515823364
[epoch:46,batch:329]:acc: 0.910417,loss:0.212016
[epoch:46,batch:359]:acc: 0.910590,loss:0.211948
[epoch:46,batch:389]:acc: 0.910337,loss:0.212572
[epoch:46,

[epoch:50,batch:59]:acc: 0.906250,loss:0.215263
[epoch:50,batch:89]:acc: 0.908681,loss:0.215111
[epoch:50,batch:119]:acc: 0.910156,loss:0.212988
[epoch:50,batch:149]:acc: 0.908750,loss:0.211304
[epoch:50,batch:179]:acc: 0.907639,loss:0.212327
[epoch:50,batch:209]:acc: 0.905952,loss:0.213000
[epoch:50,batch:239]:acc: 0.906120,loss:0.214427
[epoch:50,batch:269]:acc: 0.907060,loss:0.210884
[epoch:50,batch:299]:acc: 0.905521,loss:0.212961
[epoch:50,batch:299]: val_loss:0.362589,val_acc:0.862745,val_total:4539
[epoch:50,batch:329]:acc: 0.905492,loss:0.213446
[epoch:50,batch:359]:acc: 0.905816,loss:0.213726
[epoch:50,batch:389]:acc: 0.906891,loss:0.212952
[epoch:50,batch:419]:acc: 0.907440,loss:0.212165
[epoch:50,batch:449]:acc: 0.907500,loss:0.212449
[epoch:50,batch:479]:acc: 0.907813,loss:0.211561
[epoch:50,batch:509]:acc: 0.907721,loss:0.211789
[epoch:50,batch:539]:acc: 0.907234,loss:0.213052
[epoch:50,batch:569]:acc: 0.907785,loss:0.212420
[epoch:50,batch:599]:acc: 0.908073,loss:0.211359

In [ ]:
TrainWithRawData('../model/Inception_Resnet/2018-10-31_loss_best.pth',80)

[+] loading model... Done
min_loss is :0.361234
val_correct is 0.853933
Epoch 48/79
----------
begin lr is  0.0001
[epoch:48,batch:29]:acc: 0.858333,loss:0.403655
[epoch:48,batch:59]:acc: 0.863542,loss:0.365194
[epoch:48,batch:89]:acc: 0.863889,loss:0.361170
[epoch:48,batch:119]:acc: 0.863021,loss:0.365324
[epoch:48,batch:149]:acc: 0.865833,loss:0.361373
[epoch:48,batch:179]:acc: 0.867708,loss:0.356271
[epoch:48,batch:209]:acc: 0.862500,loss:0.360418
[epoch:48,batch:239]:acc: 0.861458,loss:0.357615
[epoch:48,batch:269]:acc: 0.860185,loss:0.358560
[epoch:48,batch:299]:acc: 0.858750,loss:0.362283
[epoch:48,batch:299]: val_loss:0.371190,val_acc:0.852390,val_total:4539
[epoch:48,batch:329]:acc: 0.860795,loss:0.360749
[epoch:48,batch:359]:acc: 0.859896,loss:0.361340
[epoch:48,batch:389]:acc: 0.858494,loss:0.361377
[epoch:48,batch:419]:acc: 0.858929,loss:0.358985
[epoch:48,batch:449]:acc: 0.857361,loss:0.362145
[epoch:48,batch:479]:acc: 0.855599,loss:0.365613
[epoch:48,batch:509]:acc: 0.8550

[epoch:50,batch:239]:acc: 0.846615,loss:0.377062
[epoch:50,batch:269]:acc: 0.850231,loss:0.373942
[epoch:50,batch:299]:acc: 0.853125,loss:0.366916
[epoch:50,batch:299]: val_loss:0.374902,val_acc:0.848425,val_total:4539
[epoch:50,batch:329]:acc: 0.851326,loss:0.373338
[epoch:50,batch:359]:acc: 0.851910,loss:0.370853
[epoch:50,batch:389]:acc: 0.850962,loss:0.370969
[epoch:50,batch:419]:acc: 0.852976,loss:0.365852
[epoch:50,batch:449]:acc: 0.854167,loss:0.364682
[epoch:50,batch:479]:acc: 0.853776,loss:0.364559
[epoch:50,batch:509]:acc: 0.854044,loss:0.364492
[epoch:50,batch:539]:acc: 0.853241,loss:0.365804
[epoch:50,batch:569]:acc: 0.853180,loss:0.365660
[epoch:50,batch:599]:acc: 0.854062,loss:0.365496
[epoch:50,batch:599]: val_loss:0.393717,val_acc:0.846883,val_total:4539
[epoch:50,batch:629]:acc: 0.854365,loss:0.366737
[epoch:50,batch:659]:acc: 0.853409,loss:0.368414
[epoch:50,batch:689]:acc: 0.853986,loss:0.367216
[epoch:50,batch:719]:acc: 0.854514,loss:0.366565
[epoch:50,batch:749]:ac

In [ ]:
def trainWithValidationData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/Inception_Resnet/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'Inception_Resnet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'Inception_Resnet', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'Inception_Resnet', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)
    

In [25]:

'''
prob:对各个类别预测的概率，这里的prob是经过softmax之后的

'''
def calibrate_probs(train_df,val_df,prob,NB_CLASS):
    calibrated_prob = np.zeros_like(prob)
    nb_train = train_df.shape[0]
    for class_ in range(NB_CLASS): # enumerate all classes 这里有61类 其他
        prior_y0_train = (train_df['disease_class'] == class_).mean() #类别为class_的先验
        prior_y1_train = 1 - prior_y0_train
        prior_y0_test=(val_df['disease_class'] ==class_).mean()
        prior_y1_test=1-prior_y0_test
        for i in range(prob.shape[0]): # enumerate every probability for a class
            predicted_prob_y0 = prob[i, class_]
            calibrated_prob_y0 = calibrate(
                prior_y0_train, prior_y0_test,
                prior_y1_train, prior_y1_test,                
                predicted_prob_y0)
            calibrated_prob[i, class_] = calibrated_prob_y0
    return calibrated_prob

def calibrate(prior_y0_train, prior_y0_test,
              prior_y1_train, prior_y1_test,
              predicted_prob_y0):
    predicted_prob_y1 = (1 - predicted_prob_y0)
    
    p_y0 = prior_y0_test * (predicted_prob_y0 / prior_y0_train)
    p_y1 = prior_y1_test * (predicted_prob_y1 / prior_y1_train)
    return p_y0 / (p_y0 + p_y1)  # normalization


'''
第二种平滑的方式


'''

def calibrate_prob2(positive_prob_train, positive_prob_test, prob):
    return (positive_prob_test * prob) / (positive_prob_test * prob + positive_prob_train * (1 - prob))

def calibrate_probs2(train_df,val_df,prob,NB_CLASS):
    calibrated_prob = np.zeros_like(prob)
    nb_train = train_df.shape[0]
    for class_ in range(NB_CLASS):
        positive_prob_train = ((train_df['disease_class'] ) == class_).mean()
        positive_prob_test =(val_df['disease_class'] ==class_).mean()
        for i in range(prob.shape[0]):
            old_p = prob[i, class_]
            new_p = calibrate_prob2(positive_prob_train, positive_prob_test, old_p)
            calibrated_prob[i, class_] = new_p
    return calibrated_prob
    

In [29]:

def getPredict(path):
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])        
    print('min_loss is :%f'%(modelParams['val_loss']))
    print('val_correct is %f'%(modelParams['val_correct']))
    lx,px=utils.predict(model,val_dataLoader)
    _, preds = torch.max(px, dim=1)
    accuracy = torch.mean((preds == lx).float())
    print('val acc on GPU is %f'%(accuracy))
    score=F.softmax(px,dim=1).cpu().numpy()
    return score,lx
#     val_pred=np.argmax(score,axis=1)
#     accurarys=np.mean(val_pred == lx.cpu().numpy())
#     print('val acc on CPU is %f'%(accurarys))
#     scoreAfter=calibrate_probs(trainDataFram,validateDataFram,score,NB_CLASS)
#     val_preAfter=np.argmax(scoreAfter,axis=1)
#     accurarysAfterCal=np.mean(val_preAfter == lx.cpu().numpy())
#     print('accurarysAfterCal is',accurarysAfterCal)
    
#     score2=calibrate_probs2(trainDataFram,validateDataFram,score,NB_CLASS)
#     val_preAfter2=np.argmax(score2,axis=1)
#     accurarysAfterCal2=np.mean(val_preAfter2 == lx.cpu().numpy())
#     print('accurarysAfterCal2 is',accurarysAfterCal2)


